# Expectation Maximization

The *Expectation Maximization (E-M) Algorithm* is an iterative approach to finding maximum likelihood estimates for latent variables (since the likelihoods we want to maximize are only indirectly available). It is comprised of an estimation step, which tries to estimate the unknown variables, and a maximization step, which then tries to optimize the parameters of the model to better explain the data.

The unknown parameters are sometimes written as $\phi$ or $\Theta$, and we can call the latent, "nuisance," variables $J$, and the observed data $U$. So, from above, the process can be roughly seen as 
$$ \Theta^* = \operatorname*{argmax}_{\Theta} \sum_{J\in\mathcal{J}^n} P(\Theta, J|U) $$ 
Since this shows us maximizing the posterior probability of parameters $\Theta$ given our data and we are summing over $J$ in order to marginalize out our latent variables (Dellaert, 2002).

This process was first rigorously defined on the exponential family, where the probability density functions take the form 
$$ f(x|\phi) = b(x)\exp(\phi t(x)^T/a(\phi))$$

where $\phi$ is a $1\times r$ parameter vector and $t(x)$ is a $1\times r$ vector of sufficient statistics for the data. Our "natural parameter" for these exponential distributions is given by some $r\times r$ linear transformation. 

To run the \textbf{E-M} algorithm on this example, we first enter the expectation step, and take $t^{(p)}=E[t(x)|y,\phi^{(p)}]$  with the $(p)$ denoting the $p^{th}$ cycle of the algorithm, trying to estimate the vector of sufficient statistics for the exponential distribution. 

The maximization step, is then taking the equation $E[t(x)|y,\phi^{(p)}]=t^{(p)}$ and we call the solution to this equation $\phi^{(p+1)}$. We then plug in $\phi^{(p+1)}$ to the expectation step and keep iterating (Dempster *et al.* 1976). 

One of the coolest parts of this algorithm is that it actually converges to a local maximum almost every time (Dempster *et al.* 1976). Dempster, Laird, and Rubin may have been slighly optimistic about how often E-M converged, due to a misuse of the triangle inequality, but C.F. Jeff Wu found and corrected that mistake, and came out with a more concrete understanding of when the algorithm converges, and to what it converges (Wu, 1983). While the algorithm may not converge to a local maximum every time, the likelihood that the parameters the algorithm optimizes each step are better estimates increases after each cycle of the algorithm. And thus, for some E-M sequence $\phi_p$, if $L(\phi_p)$ is bounded above, then E-M converges to some $L^*$.

In Dempster, Laird, and Rubin's seminal paper *Maximum Likelihood via the 'EM' Algorithm*, they enumerate the process detailed above, proved the convergence, and later on, proposed that E-M could be used to in *finite clusters*. This foreshadowed the most common usage of the algorithm, clustering, or more specifically, dividing unlabeled data into nice clusters. For example, if we know that our raw data is comprised of unique groups represented by different probability distributions, we can use the E-M algorithm to systematically change the parameters for the estimated distributions of these groups to maximize the probability that the data belongs to the proposed clustering.

## Gaussian Mixture Modeling

One of the most common usages of expectation maximization, and specifically clustering, is *Gaussian Mixture Modeling* (GMM) (Hasselblad 1966). This process is essentially assuming that each group you are trying to sort out is represented by a multivariate normal distribution. This is often a very convenient technique to use because things often actually do follow normal distributions because of the central limit theorem and because once we have clusters that are normal, it is much easier to do inference on the clusters. In industry and other settings where statisticians have to communicate their results to non-statisticians it is also helpful to be able to talk about results in the context of normal distributions.

GMMs are used to observe clusters everywhere. They are used to create customer archetypes in retail, to better understand the different ways people shop, they are used in medical scenarios in order to identify types of tumors for cancer detection.

### Example


In [ ]:
#| echo: false
from sklearn.mixture import GaussianMixture
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np
from numpy import random
from urllib.request import urlopen
from bs4 import BeautifulSoup
import seaborn as sns 
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances_argmin_min
import requests

For a flavor of the effectiveness and accuracy of the algorithm, I will make a set of five blobs of data, each with a center, and then I will show how accurately the algorithm can cluster the data into the blobs that created the underlying data.


In [ ]:
#| echo: false
#set a random seed so that we actually get clusters that kind of look like separate clusters
random.seed(195)
x, _ = make_blobs(n_samples=450, centers=5, cluster_std=1.84)
plt.figure(figsize=(8, 6))
plt.scatter(x[:,0], x[:,1])
plt.show() 

Now we fit the algorithm with the prior understanding that the data is made of five clusters of approximately normal data.


In [ ]:
#| output: false
#| echo: false
gm = GaussianMixture(n_components=5).fit(x)

gm.get_params() 

{'covariance_type': 'full',
 'init_params': 'kmeans',
 'max_iter': 100,
 'means_init': None,
 'n_components': 5,
 'n_init': 1,
 'precisions_init': None,
 'random_state': None,
 'reg_covar': 1e-06,
 'tol': 0.001,
 'verbose': 0,
 'verbose_interval': 10,
 'warm_start': False,
 'weights_init': None} 

We next fit the model and let it learn from the data, in the next plot, we plot the centers that the data came up with, and on the graph after that we will color each of the clusters the algorithm identified uniquely.


In [ ]:
#| echo: false
centers = gm.means_
plt.figure(figsize=(8, 6))
plt.scatter(x[:,0], x[:,1], label="data")
plt.scatter(centers[:,0], centers[:,1],c='r', label="centers")
plt.legend()
plt.show() 

In [ ]:
#| echo: false
pred = gm.predict(x)    

df = pd.DataFrame({'x':x[:,0], 'y':x[:,1], 'label':pred})
groups = df.groupby('label')

ig, ax = plt.subplots()
for name, group in groups:
    ax.scatter(group.x, group.y, label=name)

ax.legend()
plt.show() 

The accuracy is great! This problem wasn't terribly difficult, but it is better at understanding 'groups' of things than other classification methods in statistics and similar fields. Since many situations don't have linearly separable data, like the example above, clustering models can be a great choice over other models that are useful in other situations, such as regression models (which are sometimes used for clssification in separable cases, i.e. gentoo penguin dataset).

Consider a dataset with millions of points and possibly hundreds or thousands of dimensions. E-M allows for insights in incredibly complex problems like very sensitive anomaly detection for genetic disorders by clustering genes or proteins.

## The NBA

![](nbaPic.jpeg){width="50%"}

Basketball is changing. Players like Stephen Curry have changed perceptions around what a point guard is supposed to be, Nikola Jokic is reinventing the center position, and some teams are playing with centers who are shorter than 6'5". Another even bigger change is the advent of extremely tall players playing seemingly positionless basketball, the trend started by players such as Kevin Durant and Kristaps Porzingis, and continued by younger players like Chet Holmgren and Victor Wembanyama.

People are playing basketball differently. To effectively understand the game, the old labels of point guard, shooting guard, center, power forward, and small forward don't seem to suffice, which means that we want to find new labels for positions in order to regroup players to better understand modern roster construction.

This seems to be a problem uniquely well suited to clustering. I plan on looking at a few things, how the clusters of players in the modern NBA compare to the positions that players are assigned to. Secondly, I am curious if the NBA has become more specialized, i.e. if there are more than five positions, and players are acquired and used for more specific purposes.

In [ ]:
#| echo: false
#| output: false
urls = ['https://www.basketball-reference.com/leagues/NBA_2022_per_game.html', 'https://www.basketball-reference.com/leagues/NBA_2023_per_game.html', 'https://www.basketball-reference.com/leagues/NBA_2021_per_game.html', 'https://www.basketball-reference.com/leagues/NBA_2019_per_game.html', 'https://www.basketball-reference.com/leagues/NBA_2018_per_game.html', 'https://www.basketball-reference.com/leagues/NBA_2017_per_game.html', 'https://www.basketball-reference.com/leagues/NBA_2016_per_game.html']
def scrape_basketball_data(url):
    html = urlopen(url)
    org_html = BeautifulSoup(html)
    org_html.findAll('tr', limit=2)

    headers = [th.getText() for th in org_html.findAll('tr', limit=2)[0].findAll('th')]
    headers = headers[1:]

    rows = org_html.findAll('tr')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
    data = pd.DataFrame(player_stats, columns = headers)
    
    return data

def aggregate_data_from_urls(urls):
    all_dataframes = []
    for url in urls:
        df = scrape_basketball_data(url)
        all_dataframes.append(df)
    
    combined_data = pd.concat(all_dataframes, ignore_index=True)
    combined_data = combined_data.apply(pd.to_numeric, errors='ignore')
    
    aggregated_data = combined_data.groupby('Player', as_index=False).mean()
    return aggregated_data

df = aggregate_data_from_urls(urls)
df = df[df['MP']>=10]
df

There are a few instances of the same player showing multiple times in the dataframe since people were traded and played for different teams throughout the season, so I took the averages of all of their values to create a set of stats for the season for them.


In [ ]:
#| echo: false
#| output: false
df=df.drop(columns=['Age', 'G', 'GS', '2P', '3P', 'TRB', 'FT','PF', 'FG', 'MP'], axis=1)

So now that we have this data, I will take all of the features besides position, age, team, games played, games started (As well as some irrelevant features, i.e. those that are just linear combinations of other features) and will use them to create clusters so we can start to draw some conclusions. We should feel pretty good about modeling the clusters as gaussian since there are over eight hundred players that played in 2022 which means we should feel alright about assuming normality across each predictor, especially since we have data across 6 seasons.


In [ ]:
#| echo: false
#| output: false
X = df.copy(deep=True)

X.replace('', np.nan, inplace=True)
# Drop rows with NaN values
X.dropna(inplace=True)
names = X["Player"]
X = X.drop("Player", axis = 1)

In [ ]:
#| echo: false
n_components = np.arange(1, 15)
models = [GaussianMixture(n, covariance_type='full', random_state=0).fit(X)
          for n in n_components]

plt.plot(n_components, [m.bic(X) for m in models], label='BIC')
plt.plot(n_components, [m.aic(X) for m in models], label='AIC')
plt.legend(loc='best')
plt.xlabel('n_components');

This plot shows metrics *Bayesian Information Criterion* (BIC) and *Aikake Information Criterion* (AIC) plotted against the number of clusters to see which number of clusters would most likely produce the data while also penalizing overfitting. The lower the value the better fit the model is for the data. Both of the metrics are based on the likelihood function for the mixture models and the main difference between them is that BIC punishes models with more parameters more than AIC, as we can see from the plot, since the BIC is minimized at right around 4 clusters and AIC continues to decrease as the number of components reaches 15. The AIC is equal to $2k-2\ln(L)$ and the BIC is equal to $k\ln(n)-2\ln(L)$ where $k$ is the number of parameters. So as the likelihood that the model proposed (dependent on the number of clusters) has a higher likelihood of explaining the data, the AIC and BIC both decrease. 

This graph essentially says that the model that explains the data the best without overfitting has 4 clusters (potentially saying that the NBA is *less* specialized than in the past).


In [ ]:
#| echo: false
model = GaussianMixture(n_components=4, random_state=0).fit(X)

In [ ]:
#| echo: false
#| output: false
preds = pd.Series(model.predict(X))
X["Cluster"] = preds.values
X["Name"] = names
Z = X.copy(deep=True)

In [ ]:
#| echo: false
#| output: false
Y = X.drop('Cluster', axis = 1)
cluster_assignments = model.predict(Y)

# Get the cluster centers from the fitted model
cluster_centers = model.means_  # This will give the centers of each cluster

# Calculate distances between each player and the cluster centers
distances_to_centers = np.linalg.norm(Y - cluster_centers[cluster_assignments], axis=1)

# Find the indices of players closest to each cluster center
closest_players_indices = np.argmin(distances_to_centers, axis=0)

# Get the details of players closest to each cluster center
closest_players = Y.iloc[closest_players_indices]

In [ ]:
#| echo: false
#| output: false
X_numeric = X.select_dtypes(include='number')

# Scale the numeric columns
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numeric)

# Convert back to DataFrame (keeping the column names)
X_scaled_df = pd.DataFrame(X_scaled, columns=X_numeric.columns)

# Add the cluster assignments to your scaled data
X_scaled_df['Cluster'] = cluster_assignments

# Melt the DataFrame to create a long-form dataset for violin plot
melted_data_scaled = X_scaled_df.melt(id_vars='Cluster', var_name='Statistic', value_name='Value')

# Plotting separate violin plots for each statistic
g = sns.FacetGrid(melted_data_scaled, col='Statistic', col_wrap=3, height=4)
g.map(sns.violinplot, 'Cluster', 'Value', palette='Set3')
g.set_xticklabels(rotation=45)
g.set_titles('{col_name}')
plt.tight_layout()
plt.show()

In [ ]:
#| echo: false
#| output: false
"""X_numeric = X.select_dtypes(include='number')

# Scale the numeric columns
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numeric)

# Convert back to DataFrame (keeping the column names)
X_scaled_df = pd.DataFrame(X_scaled, columns=X_numeric.columns)

# Add the cluster assignments to your scaled data
X_scaled_df['Cluster'] = cluster_assignments

# Iterate through each statistic and create individual violin plots
for stat in X_numeric.columns:
    # Create melted data for the current statistic
    melted_data_stat = X_scaled_df.melt(id_vars='Cluster', value_vars=stat, var_name='Statistic', value_name='Value')

    # Plot the violin plot for the current statistic
    plt.figure(figsize=(6, 4))  # Adjust the figure size if needed
    sns.violinplot(x='Cluster', y='Value', data=melted_data_stat, palette='Set3')
    plt.title(f'Violin Plot for {stat}')
    plt.xlabel('Cluster')
    plt.ylabel('Scaled Value')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()"""
    #this was used to generate the violin plots I threw all over the end of this paper

### Results

The following violin plots (which have been scaled) show the distributions for each cluster for each statistic. 

::: {layout-nrow=1}
![](2PA.png)

![](3PA.png)
:::
Cluster 2 obviously attempts what looks like significantly more two point shots and three point shots. Cluster 1 attempts the second most 2 pointers while also completing the least three pointers which contrasts with Cluster 3 which attempts the least two pointers while attempting more three pointers compared to the other clusters.

:::{layout-nrow=1}
![](FTA.png)

![](PTS.png)
:::

From the first two plots, it makes sense that Cluster 2, on average, scores the most points because they attempt the most 2PT and 3PT shots. Clusters 0 and 1 score the next most points on average, probably due to the larger number of 2PT attempts by Cluster 1 and the larger number of 3PT attempts from Cluster 0. Cluster 0 has a much larger mean number of free throws attempted while Cluster 3 has a significantly lower number of free throws attempted.

This leads us to start to see a picture being painted about these players, Cluster 2 in particular has the ball a lot of the time, they score the most points, they shoot the most free throws, they are high usage players. 

:::{layout-nrow=1}
![](TOV.png)

![](AST.png)
:::

Cluster 2 also has the most turnovers on average, 


In [ ]:
closest_players_indices, _ = pairwise_distances_argmin_min(X_scaled, cluster_centers)

# Assuming 'X' has a 'Player' column
closest_players = X.loc[closest_players_indices]

# Print the players closest to each cluster center
for i, player_index in enumerate(closest_players_indices):
    player_name = X.loc[player_index, 'Player']
    print(f"Cluster {i} Center: Player with stats closest: {player_name}")